In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from pathlib import Path
from collections import defaultdict
from itertools import product
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations,permutations
from sklearn.tree import *
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
import random
from math import floor
import math
import random

In [2]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, sampler
from matplotlib import pyplot as plt
from albumentations import (GaussianBlur,RandomCrop,MedianBlur,ElasticTransform,Flip,HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.pytorch.transforms import ToTensor
import cv2
import pyarrow as pa
import pyarrow.parquet as pq
from torchvision import datasets,models,transforms
import sklearn
from tqdm import tqdm
import time

In [3]:
data_path = Path("/kaggle/input/abstraction-and-reasoning-challenge")
train_path = data_path/'training'
test_path = data_path/'test'
evaluation_path = data_path/'evaluation'

In [4]:
import json
import os
from os.path import join as path_join


def load_data(path):
    tasks = pd.Series()
    for file_path in os.listdir(path):
        task_file = path_join(path, file_path)

        with open(task_file, 'r') as f:
            task = json.load(f)

        tasks[file_path[:-5]] = task
    return tasks

In [5]:
train_tasks = load_data('../input/abstraction-and-reasoning-challenge/training/')
evaluation_tasks = load_data('../input/abstraction-and-reasoning-challenge/evaluation/')
test_tasks = load_data('../input/abstraction-and-reasoning-challenge/test/')
train_tasks.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


0b148d64    {'train': [{'input': [[8, 8, 8, 8, 8, 0, 8, 8,...
54d9e175    {'train': [{'input': [[0, 0, 0, 5, 0, 0, 0, 5,...
a8c38be5    {'train': [{'input': [[5, 5, 5, 0, 0, 0, 0, 0,...
ce9e57f2    {'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0,...
57aa92db    {'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0,...
dtype: object

In [6]:
## Plotting functions

import matplotlib.pyplot as plt
from matplotlib import colors

def plot_task(task,j=0,k=0):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, 4, figsize=(15,15))
    axs[0].imshow(task['train'][j]['input'], cmap=cmap, norm=norm)
    axs[0].axis('off')
    axs[0].set_title('Train Input')
    axs[1].imshow(task['train'][j]['output'], cmap=cmap, norm=norm)
    axs[1].axis('off')
    axs[1].set_title('Train Output')
    axs[2].imshow(task['test'][k]['input'], cmap=cmap, norm=norm)
    axs[2].axis('off')
    axs[2].set_title('Test Input')
    axs[3].imshow(task['test'][k]['output'], cmap=cmap, norm=norm)
    axs[3].axis('off')
    axs[3].set_title('Test Output')
    plt.tight_layout()
    plt.show()

In [7]:
def plot_images(images):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    m = len(images)
    fig, axs = plt.subplots(1, m, figsize=(15,15))
    for i in range(m):
        axs[i].imshow(images[i], cmap=cmap, norm=norm)
        axs[i].axis('off')
        axs[i].set_title('Image' + str(i))
    plt.tight_layout()
    plt.show()

DSL

In [8]:
class fund_unit:
    
    def __init__(self,colour,position,ambient = -1):
        self.colour = colour
        self.position = position #list of places
        l1 = [x[0] for x in position]
        l2 = [x[1] for x in position]
        a = min(l1)
        corners = get_corners(position)
        self.top_right = corners['top_right']
        self.top_left = corners['top_left']
        self.bottom_left = corners['bottom_left']
        self.bottom_right = corners['bottom_right']
        self.colour = colour
        self.ambient = ambient
        self.right = corners['right']
        self.left = corners['left']
        self.bottom = corners['bottom']
        self.top = corners['top']

In [9]:
def create(colour,position):
    unit = fund_unit(colour,position)
    return(unit)

## Attribute functions

In [10]:
#units is a list
def get_colour_dict(units):
    colours = {}
    for j in range(10):
        colours[j] = []
        for unit in units:
            if unit.colour == j:
                colours[j].append(unit)
        colours[j].sort(key = lambda unit: (unit.top_left[0],unit.top_left[1]))
    return(colours)    

In [11]:
def get_corners(position):
    l1 = [x[0] for x in position]
    l2 = [x[1] for x in position]
    a = min(l1)
    l3 = []
    for x in position:
        if x[0] == a:
            l3.append(x[1])
    b = min(l3)
    c = max(l3)
    corners = {'top_left' : np.array([a,b]),'top_right' : np.array([a,c])}
    a = max(l1)
    for x in position:
        if x[0] == a:
            l3.append(x[1])
    b = min(l3)
    c = max(l3)
    corners['bottom_left'] = np.array([a,b])
    corners['bottom_right'] = np.array([a,c])
    # get the coordinates of the top cell
    a = min(l1)
    l3 = []
    for x in position:
        if x[0] == a:
            l3.append(x[1])
    l3.sort()        
    if len(l3)%2 == 1:
        m = (len(l3)-1)//2
    else:
        m = len(l3)//2
    corners['top'] = np.array([a,l3[m]])
    #coordinates of the bottom cell
    a = max(l1)
    l3 = []
    for x in position:
        if x[0] == a:
            l3.append(x[1])
    l3.sort()        
    if len(l3)%2 == 1:
        m = (len(l3)-1)//2
    else:
        m = len(l3)//2
    corners['bottom'] = np.array([a,l3[m]])    
    #coordinates of the left cell
    a = min(l2)
    l3 = []
    for x in position:
        if x[1] == a:
            l3.append(x[0])
    l3.sort()        
    if len(l3)%2 == 1:
        m = (len(l3)-1)//2
    else:
        m = len(l3)//2
    corners['left'] = np.array([l3[m],a])
    #coordinates of the right cell 
    a = max(l2)
    l3 = []
    for x in position:
        if x[1] == a:
            l3.append(x[0])
    l3.sort()        
    if len(l3)%2 == 1:
        m = (len(l3)-1)//2
    else:
        m = len(l3)//2
    corners['right'] = np.array([l3[m],a])
    return(corners)

In [12]:
def get_size_list(units):
    units.sort(key = lambda u : len(u.position))
    return(units)    

In [13]:
# returns chunks of unit1 which are copies of unit2 after moving it. 
def pattern_matching(unit1,unit2,with_colour = False):
    if with_colour and not unit1.colour == unit2.colour:
        return([])
    else:
        l = []
        matches = []
        y = unit2.position[0]
        for x in unit1.position:
            l.append(x-y)
        s1 = [(a[0],a[1]) for a in unit1.position]        
        s1 = set(s1)
        for z in l:
            p2 = [y + z for y in unit2.position]
            p2 = [(a[0],a[1]) for a in p2]
            p2 = set(p2)
            if p2.issubset(s1):
                p2 = list(p2)
                p2 = [np.array([a[0],a[1]]) for a in p2]
                matches.append([p2,z])
        matches.sort(key = lambda a : abs(a[1][0]) + abs(a[1][1]))
        return(matches)                

In [14]:
# sorts a list of units by size
def sort_by_size(l):
    l.sort(key = lambda u : len(u.position))
    return(l)

In [15]:
def sort_by_position(l,base = 0):
    if base != 3 and base !=7:
        l.sort(key = lambda u : (give_base(u,base)[0],give_base(u,base)[1]))
    if base == 7:
        l.sort(key = lambda u : (give_base(u,base)[1],give_base(u,base)[0]))
    if base == 3:
        l.sort(key = lambda u : (-give_base(u,base)[1],give_base(u,base)[0]))
    return(l)

def give_base(unit,base):
    d = {}
    d[0] = unit.top_left
    d[1] = unit.top
    d[2] = unit.top_right
    d[3] = unit.right
    d[4] = unit.bottom_right
    d[5] = unit.bottom 
    d[6] = unit.bottom_left
    d[7] = unit.left
    return(d[base])

## Change unit in place / Get units 

In [16]:
def change_colour(l,c):
    for unit in l:
        unit.colour = c
    return(l)

In [17]:
def get_units(img,with_diag = False):
    img = np.array(img)
    r,c = img.shape
    ambient = img.shape
    E = {}
    V = []
    for i in range(r):
        for j in range(c):
            V.append((i,j))
            E[(i,j)] = [] 
    for i in range(r):
        for j in range(c):
            if i-1 >= 0 and img[i-1,j] == img[i,j]:
                    E[(i,j)].append((i-1,j))
            if i+1 <= r-1 and img[i+1,j] == img[i,j]:
                E[(i,j)].append((i+1,j))
            if j-1 >= 0 and img[i,j-1] == img[i,j]:
                E[(i,j)].append((i,j-1))
            if j+1 <= c-1 and img[i,j+1] == img[i,j]:
                E[(i,j)].append((i,j+1))
            if with_diag:    
                if i-1 >= 0 and j-1 >= 0 and img[i-1,j-1] == img[i,j]:
                    E[(i,j)].append((i-1,j-1))
                if i-1 >= 0 and j+1 <= c-1 and img[i-1,j+1] == img[i,j]:
                    E[(i,j)].append((i-1,j+1))
                if i+1 <= r-1 and j-1 >= 0 and img[i+1,j-1] == img[i,j]:
                    E[(i,j)].append((i+1,j-1))
                if i+1 <= r-1 and j+1 <= c-1 and img[i+1,j+1] == img[i,j]:
                    E[(i,j)].append((i+1,j+1))
    parent = DFS(V,E)
    units = []
    for v in parent.keys():
        if len(parent[v]) > 0:
            temp = [np.array([x[0],x[1]]) for x in parent[v]]
            unit = fund_unit(img[v],temp,ambient)
            units.append(unit)
    units.sort(key = lambda unit: (unit.top_left[0],unit.top_left[1]))        
    return(units)

In [18]:
def offset(coordinates,a,b):
    new_x = coordinates[0] + a
    new_y = coordinates[1] + b
    return(np.array([new_x,new_y]))

In [19]:
def DFS(V,E):
    colour = {}
    parent = {}
    for v in V:
        colour[v] = 0
        parent[v] = []
    for v in V:
        if colour[v] == 0:
            DFS_search(v,E,parent[v],colour)
    return(parent)        

In [20]:
def DFS_search(v,E,l,colour):
    l.append(v)
    colour[v] = 1
    for e in E[v]:
        if colour[e] == 0:
            DFS_search(e,E,l,colour)
    colour[v] = 2    

In [21]:
# f returns a list. We want it to act on every element of a list
def do_for_list(l,f):
    t = []
    for x in l:
        t = t + f(x)
    return(t)

## Check for contractibility and blank space

In [22]:
def get_genus(unit,with_diag = False):
    r,c = unit.ambient
    l = boundary_img(r,c)
    if check_belongs_to(l,unit.position):
        is_a_boundary = True
    else:
        is_a_boundary = False
    #i = 0
    #count = 0
    #while check_belongs_to([l[i]],unit.position):
    #    count += 1
    #    i += 1
    #    if i > len(l)-1:
    #        break
    #if count < 2*c + 2*r - 4:
    #    start_key = l[i]
    #    is_a_boundary = False
    #else:
    #    is_a_boundary = True
    u = fund_unit(1,unit.position,unit.ambient)
    img = concat([u],u.ambient)
    units = get_units(img,with_diag)
    enclosed_area = []
    g = 0 
    for v in units:
        if check_empty_intersection(l,v.position) and check_empty_intersection(v.position,unit.position):
            g += 1
    return(g)  

In [23]:
def check_contractible(unit,with_diag = False):
    g = get_genus(unit,with_diag)
    if g == 0:
        return(True)
    else:
        return(False)

In [24]:
def get_non_contractible(units,with_diag = False,with_background = False):
    non_contract = []
    units = sort_by_size(units)
    if not with_background and units[-1] == 0:
        units = units[:-1]
    for u in units:
        if not check_contractible(u,with_diag):
            non_contract.append(u)
    return(non_contract)   

In [25]:
def get_up_to_homotopy(units):
    t = []
    for u in units:
        if not check_contractible(u):
            t.append(u)
    return(t)        

## Check grid

In [26]:
def check_identical(l):
    u0 = l[0]
    t = 0
    for u in l:
        if len(u.position) == len(u0.position):
            p = pattern_matching(u,u0)
            if len(p) >= 1:
                t += 1
    if t == len(l):
        return(True)
    else:
        return(False)

In [27]:
def check_grid_new(img,get_diag = False):
    img = np.array(img)
    ambient = np.array(img).shape
    units = sort_by_size(get_units(img,get_diag))
    for u in units:
        c = 0
        v = []
        for w in units:
            if w != u:
                v.append(w)
        r = {}
        for w in v:
            x = w.top_left
            if x[0] in r.keys():
                r[x[0]].append(w)
            else:
                r[x[0]] = [w]
        temp = []
        for k in r.keys():
            temp.append(len(r[k]))
        if len(set(temp)) > 1:
            c = 1 
        for k in r.keys():
            if not check_identical(r[k]):
                c = 1
        if c == 0:
            return([u])
    return([]) 

In [28]:
def check_grid(img,get_diag = False):
    img = np.array(img)
    ambient = np.array(img).shape
    units = sort_by_size(get_units(img,get_diag))
    background = units[-1]
    u0 = units[0]
    c = 0
    for u in units[:-1]:
        if len(u0.position) == len(u.position):
            l = pattern_matching(u0,u)
            if len(l) > 0:
                c += 1
    if c == len(units)-1:
        return(True)
    else:
        return(False)    

In [29]:
# background is the unit of largest size

def get_background(img,get_diag = False):
    img = np.array(img)
    ambient = np.array(img).shape
    units = get_units(img,get_diag)
    background = sort_by_size(units)[-1]
    return(background)

## collecting units that are surrounded

In [30]:
def check_actual_coord(x,y,r,c):
    if x >= 0 and x <= r-1 and y >= 0 and y <= c-1:
        return(True)
    else:
        return(False)

In [31]:
def udlr(x,r,c):
    a,b = x[0],x[1]
    l = []
    x_up = [a-1,b]
    x_down = [a+1,b]
    x_right = [a,b+1]
    x_left = [a,b-1]
    t = [x_up,x_down,x_right,x_left]
    for i in range(4):
        if check_actual_coord(t[i][0],t[i][1],r,c):
            l.append(np.array(t[i]))
    return(l)       

In [32]:
def is_surrounded(u1,u2):
    r,c = u1.ambient
    val = True
    for x in u1.position:
        t = udlr(x,r,c)
        m = len(t)
        if m < 4:
            val = False
        for i in range(m):
            if not check_belongs_to([t[i]],u1.position):
                if not check_belongs_to([t[i]],u2.position):
                    val = False
    return(val)     

In [33]:
# Returns a dict - surrounded, where surrounded[u] is the list of units v surrounded by u.

def get_surrounded(units):
    surrounded = {}
    for u in units:
        surrounded[u] = []
        for v in units:
            if is_surrounded(v,u):
                surrounded[u].append(v)
    return(surrounded)    

In [34]:
def surrounded_list(units):
    surrounded = []
    d = get_surrounded(units)
    l = []
    for k in d.keys():
        l = l + d[k]
    return(l)    

## Get Shapes which are contained in other shapes

In [35]:
def boundary_img(r,c):
    l = []
    for j in range(c):
        l.append(np.array([0,j]))
    for j in range(r):
        l.append(np.array([j,c-1]))
    for j in range(c):
        l.append(np.array([r-1,j]))
    for j in range(r):
        l.append(np.array([j,0]))
    return(l)    

In [36]:
def get_intersection(l1,l2):
    l11 = [(x[0],x[1]) for x in l1]
    l22 = [(x[0],x[1]) for x in l2]
    l = list(set(l11)&set(l22))
    l = [np.array(x) for x in l]
    return(l)

In [37]:
def check_empty_intersection(l1,l2):
    l = get_intersection(l1,l2)
    if len(l) > 0:
        return(False)
    else:
        return(True)

In [38]:
# Assume the container is not black

def get_enclosed_area(unit):
    r,c = unit.ambient
    l = boundary_img(r,c)
    i = 0
    count = 0
    while check_belongs_to([l[i]],unit.position):
        count += 1
        i += 1
        if i > len(l)-1:
            break
    if count < 2*c + 2*r - 4:
        start_key = l[i]
        is_a_boundary = False
    else:
        is_a_boundary = True
    u = fund_unit(1,unit.position,unit.ambient)
    img = concat([u],u.ambient)
    units = get_units(img)
    enclosed_area = []
    if is_a_boundary:
        for v in units:
            if check_belongs_to([v.top_left],[u.top_left]):
                enclosed_area += v.position                
    if len(units) > 2 and not is_a_boundary:
        for v in units:
            if not check_belongs_to([v.top_left],[u.top_left]) and check_empty_intersection(l,v.position):
                enclosed_area += v.position
    return(enclosed_area)            

In [39]:
def contained_list(units):
    contained = []
    enclosed_area = []
    for u in units:
        if u.colour != 0:
            enclosed_area += get_enclosed_area(u)
    for v in units:
        if check_belongs_to(v.position,enclosed_area):
            contained.append(v)
    return(contained)                   

In [40]:
def contained_in(units):
    d = {}
    l = detect_regular_non_contractible(units)
    for u in l:
        d[u] = []
        x = u.top_left
        y = u.bottom_right
        l1 = list(range(x[0],y[0]+1))
        l2 = list(range(x[1],y[1]+1))
        temp = list(itertools.product(l1,l2))
        p = []
        for t in temp:
            p.append(np.array([t[0],t[1]]))
        pdiff = []    
        for v in p:
            r0 = 0
            for z in u.position:
                if v[0] == z[0] and v[1] == z[1]:
                    r0 += 1
            if r0 == 0:
                pdiff.append(v)
        udiff = fund_unit(0,pdiff,u.ambient)
        for v in units:
            if check_belongs_to(v.position,u.position):
                d[u].append(v)
    return(d)            

## Extracting simple shapes

In [41]:
def is_regular(unit):
    x = unit.top_left
    y = unit.bottom_right
    l1 = list(range(x[0],y[0]+1))
    l2 = list(range(x[1],y[1]+1))
    temp = list(itertools.product(l1,l2))
    p = []
    for t in temp:
        p.append(np.array([t[0],t[1]]))
    u0 = fund_unit(0,p,unit.ambient)
    if len(p) == len(unit.position):
        l = pattern_matching(u0,unit)
        if len(l) > 0:
            if len(l1) == len(l2):
                return('square')
            if len(l1) == 1 and len(l2) > len(l1):
                return('vertical line')
            if len(l2) == 1 and len(l1) > len(l2):
                return('horizontal line')
            if len(l1) > len(l2):
                return('horizontal rectangle')
            if len(l2) > len(l1):
                return('vertical rectangle')
    return('-1')        

In [42]:
def get_regular_shapes(units):
    d = {}
    d['square'] = []
    d['vertical line'] = []
    d['horizontal line'] = []
    d['horizontal rectangle'] = []
    d['vertical rectangle'] = []
    d['-1'] = []
    for u in units:
        d[is_regular(u)].append(u)
    return(d)   

In [43]:
# returns a dict whose keys are elements of the list 'units' and whose values are equivalence classes coded as integers or the list of equivalent shapes.  

def shape_equivalences(units,return_lists = True,remove_background = False): 
    #img = np.array(img)
    #ambient = np.array(img).shape
    units = sort_by_size(units)
    if remove_background:
        units = units[:-1]    
    d = {}
    d[units[0]] = []
    is_key = {}
    for u in units:
        is_key[u] = False
    for u in units:
        c = 0
        for v in units:
            if is_key[v]:
                if len(u.position) == len(v.position):
                    l = pattern_matching(u,v)
                    if len(l) > 0:
                        c += 1
                        d[v].append(u)
        if c == 0:
            d[u] = [u]
            is_key[u] = True
    m = len(d.keys())
    shape_equivalences = {}
    for j,key in enumerate(d.keys()):
        for u in d[key]:
            shape_equivalences[u] = j
    if return_lists:
        return(d)
    else:
        return(shape_equivalences)

In [44]:
def get_task_num(img,task):
    m = len(task['train'])
    for i in range(m):
        if np.all(img == task['train'][i]['input']):
            return(i)

In [45]:
# Returns list of units of an image which also occur in all other images of the training set. num = -1 when img is the test image

def common_shapes_across_images(img,task,img_units,num):
    #if num == -1:
    #    num = get_task_num(img,task)
    m = len(task['train'])
    all_units = []
    task_number = {}
    for i in range(m):
        if i != num:
            units = get_units(task['train'][i]['input'])
        else:
            units = img_units
        for u in units:
            task_number[u] = i
        all_units += units
    if num == -1:
        units = img_units
        all_units += img_units
        for u in units:
            task_number[u] = i
    d = shape_equivalences(all_units) 
    shapes_across_images = []
    for x in d.keys():
        l = [task_number[y] for y in d[x]]
        if len(set(l)) == m:
            shapes_across_images.append(x)
    final_list = []
    for x in shapes_across_images:
         for u in d[x]:
                if task_number[u] == num:
                    final_list.append(u)
    return(final_list)       

In [46]:
def shape_equivalences_counts(units,remove_background = False):
    d = {}
    s = shape_equivalences(units,True,remove_background)
    for x in s.keys():
        m = len(s[x])
        if m in d.keys():
            d[m] += s[x]
        else:
            d[m] = s[x]
    return(d)      

In [47]:
# checks if l1 is a subset of l2. Both are lists of np arrays of length 2.
def check_belongs_to(l1,l2):
    r = 0 
    for z in l1:
        for x in l2:
            if x[0] == z[0] and x[1] == z[1]:
                r += 1
    if r == len(l1):
        return(True)
    else:
        return(False)

In [48]:
def check_this_please(u,x1,x2):
    l1 = list(range(x1[0],x2[0]+1))
    l2 = []
    for t in l1:
        l2.append(np.array([t,x1[1]]))
    if check_belongs_to(l1,u.position):
        count = 1
    else:
        count = 0
    return(count)  

In [49]:
import itertools
# selects the regular non contractible elements of units i.e. those elements of the list units whose hole is a regular object. 

def detect_regular_non_contractible(units):
    #img = np.array(img)
    #ambient = np.array(img).shape
    units = sort_by_size(units)
    regular_non_contractible = []
    for u in units:
        x = u.top_left
        y = u.bottom_right
        l1 = list(range(x[0],y[0]+1))
        l2 = list(range(x[1],y[1]+1))
        temp = list(itertools.product(l1,l2))
        p = []
        for t in temp:
            p.append(np.array([t[0],t[1]]))
        pdiff = []    
        for v in p:
            r0 = 0
            for z in u.position:
                if v[0] == z[0] and v[1] == z[1]:
                    r0 += 1
            if r0 == 0:
                pdiff.append(v)
        if len(pdiff) > 0:        
            udiff = fund_unit(0,pdiff,u.ambient) 
            u0 = fund_unit(0,p,u.ambient)
            s = is_regular(udiff)
            if s != '-1':
                count = 0
                x1 = udiff.top_left
                x2 = udiff.top_right
                l1 = list(range(x1[1],x2[1]+1))
                l2 = []
                for t in l1:
                    l2.append(np.array([x1[0]-1,t]))
                if check_belongs_to(l2,u.position):
                    count += 1
                x1 = udiff.top_right
                x2 = udiff.bottom_right
                l1 = list(range(x1[0],x2[0]+1))
                l2 = []
                for t in l1:
                    l2.append(np.array([t,x1[1]+1]))
                if check_belongs_to(l2,u.position):
                    count += 1 
                x1 = udiff.bottom_left
                x2 = udiff.bottom_right
                l1 = list(range(x1[1],x2[1]+1))
                l2 = []
                for t in l1:
                    l2.append(np.array([x1[0]+1,t]))
                if check_belongs_to(l2,u.position):
                    count += 1    
                x1 = udiff.top_left
                x2 = udiff.bottom_left
                l1 = list(range(x1[0],x2[0]+1))
                l2 = []
                for t in l1:
                    l2.append(np.array([t,x1[1]-1]))
                if check_belongs_to(l2,u.position):
                    count += 1
                if count == 4:
                    regular_non_contractible.append(u)
    return(regular_non_contractible)    

## Filter by size

In [50]:
def filter_by_size(units):
    f = {}
    for j in range(1,7):
        f[j] = []
    for u in units:
        if len(u.position) < 6:
            f[len(u.position)].append(u)
        else:
            f[6].append(u)
    return(f)        

## Assembling image from units

In [51]:
def concat(l,ambient,background = 0):
    r,c = ambient
    a = np.full(ambient,background)
    for x in l:
        for z in x.position:
            if z[0] >= 0 and z[0] <= r-1 and z[1] >= 0 and z[1] <= c-1:
                a[z[0],z[1]] = x.colour
    return(a)      

# Get easier sub-collection of the tasks

In [52]:
def input_output_shape_is_same(task):
    return all([np.array(el['input']).shape == np.array(el['output']).shape for el in task['train']])

def get_equals(task_series):
    l = []
    for j in task_series.index:
        if input_output_shape_is_same(task_series[j]):
            l.append(j)
    return(l) 

In [53]:
l_input_output_same_size = get_equals(train_tasks)
l_input_output_same_size.sort()

In [54]:
# convert a list of np arrays to tuples
def convert_to_tuple(t):
    s = [(a[0],a[1]) for a in t]
    return(s)

In [55]:
# count those images where one must only colour match
#l_colour_match_no_diag = []
#for index in l_input_output_same_size:
#    task = train_tasks[index]
#    c = 0
#    for example in task['train']:
#        units_input = get_units(np.array(example['input']))
#        units_output = get_units(np.array(example['output']))
#        if len(units_input) == len(units_output):
#            sub_count = 0
#            for u in units_input:
#                s1 = set(convert_to_tuple(u.position))
#                for v in units_output:
#                    s2 = set(convert_to_tuple(v.position))
#                    if s1 == s2:
#                        sub_count += 1
#            if sub_count == len(units_input):
#                c += 1
#        if c == len(task['train']):        
#            l_colour_match_no_diag.append(index) 

## Lists

In [56]:
# outputs a dict with the following encoding : 
# 0 --> all units
# 1 --> non contractible objects
# 2 --> surrounded_list
# 3 --> contained_in list
# 4 --> squares 
# 5 --> vertical_line (not using for additional features)
# 5.5 --> line
# 6 --> horizontal line (not using for additional features)
# 7 --> horizontal rectangle (not using for additional features)
# 7.5 --> rectangle
# 8 --> vertical rectangle (not using for additional features)
# 9 --> -1 (irregular shape)
# 10 --> regular shapes (union of 14,15,16,17,18) 
# 11 --> regular non contractible units
# 12 --> List of shapes with no doubles
# 13 --> shapes which are doubled
# 14 --> Shapes which are trebled
# 15 --> Shapes that appear in all train inputs (not using for additional features)
# 16 --> Shapes of size 1
# 17 --> Shapes of size 2
# 18 --> Shapes of size 3
# 19 --> Shapes of size 4
# 20 --> Shapes of size 5
# 21 --> Shapes of size greater than 5
# All negative keys are complements of the corresponding positive keys

def get_lists(img,task = None,num = -1,with_diag = False):
    list_dict = {}
    img = np.array(img)
    ambient = np.array(img).shape
    units = get_units(img,with_diag)
    list_dict[0] = units
    #c_dict = get_colour_dict(units)
    #for i in range(10):
    #    list_dict[i+1] = c_dict[i]
    #t = get_up_to_homotopy(units)
    list_dict[1] = get_non_contractible(units,with_diag,False)
    list_dict[2] = surrounded_list(units)
    list_dict[3] = contained_list(units)
    shape_d = get_regular_shapes(units)
    list_dict[4] = shape_d['square']
    #list_dict[5] = shape_d['vertical line']
    #list_dict[6] = shape_d['horizontal line']
    list_dict[5.5] = shape_d['vertical line'] + shape_d['horizontal line']
    #list_dict[7] = shape_d['horizontal rectangle']
    #list_dict[8] = shape_d['vertical rectangle']
    list_dict[7.5] = shape_d['horizontal rectangle'] + shape_d['vertical rectangle']
    list_dict[9] = shape_d['-1']
    list_dict[10] = shape_d['square'] + shape_d['vertical line'] + shape_d['horizontal line'] + shape_d['horizontal rectangle'] + shape_d['vertical rectangle']
    #for count,i in enumerate(range(14,21)):
    #    d_temp = get_colour_dict(list_dict[i])
    #    t = extract_lists_from_dict(d_temp)
    #    for j in range(10):
    #        list_dict[21+(count*10)+j] = t[j]
    list_dict[11] = detect_regular_non_contractible(units)
    shape_counts = shape_equivalences_counts(units,False)
    for i in range(1,4):
        if i not in shape_counts.keys():
            shape_counts[i] = []
    for i in range(1,4):
        list_dict[11+i] = shape_counts[i]
    #list_dict[15] = common_shapes_across_images(img,task,units,num)
    f = filter_by_size(units)
    for j in range(1,7):
        list_dict[15+j] = f[j]
    #list_dict = get_list_dict_complements(units,list_dict) 
    return(units,list_dict)    

In [57]:
# task_number == -1 if test
def additional_X_features(img,task = None,task_number = -1,with_diag = False):
    img = np.array(img)
    units,list_dict = get_lists(img,task,task_number,with_diag)
    d = {}
    r,c = img.shape
    bckg = sort_by_size(units)[-1]
    for i in range(r):
        for j in range(c):
            features = []
            if bckg.colour == 0:
                if check_belongs_to([np.array([i,j])],bckg.position):
                    features.append(1)
                else:
                    features.append(0) 
            else:
                features.append(0)
            for key in list_dict.keys():
                if key > 0:
                    temp = 0
                    if len(list_dict[key]) > 0:
                        t = sort_by_position(list_dict[key],0)
                        for idx,u in enumerate(t):
                            if check_belongs_to([np.array([i,j])],u.position):
                                temp = idx+1
                    features.append(temp)
            d[(i,j)] = features
    return(d)        

## Decision Tree

In [58]:
def getiorc(pair):
    inp = pair["input"]
    return pair["input"],pair["output"],len(inp),len(inp[0])
    
def getAround(i,j,inp,size=1):
    #v = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    r,c = len(inp),len(inp[0])
    v = []
    sc = [0]
    for q in range(size):
        sc.append(q+1)
        sc.append(-(q+1))
    for idx,(x,y) in enumerate(product(sc,sc)):
        ii = (i+x)
        jj = (j+y)
        v.append(-1)
        if((0<= ii < r) and (0<= jj < c)):
            v[idx] = (inp[ii][jj])
    return v

def getDiagonal(i,j,r,c):
    return
        
    
def getX(inp,i,j,size):
    z = []
    n_inp = np.array(inp)
    z.append(i)
    z.append(j)
    r,c = len(inp),len(inp[0])
    for m in range(5):
        z.append(i%(m+1))
        z.append(j%(m+1))
    z.append(i+j)
    z.append(i*j)
#     z.append(i%j)
#     z.append(j%i)
    z.append((i+1)/(j+1))
    z.append((j+1)/(i+1))
    z.append(r)
    z.append(c)
    z.append(len(np.unique(n_inp[i,:])))
    z.append(len(np.unique(n_inp[:,j])))
    arnd = getAround(i,j,inp,size)
    z.append(len(np.unique(arnd)))
    z.extend(arnd)
    return z

def getXy(inp,oup,size,features):
    x = []
    y = []
    r,c = len(inp),len(inp[0])
    for i in range(r):
        for j in range(c):
            x.append(getX(inp,i,j,size)+features[(i,j)])
            y.append(oup[i][j])
    return x,y
    
def getBkgColor(task_json):
    color_dict = defaultdict(int)
    
    for pair in task_json['train']:
        inp,oup,r,c = getiorc(pair)
        for i in range(r):
            for j in range(c):
                color_dict[inp[i][j]]+=1
    color = -1
    max_count = 0
    for col,cnt in color_dict.items():
        if(cnt > max_count):
            color = col
            max_count = cnt
    return color


## Augmentation + Model

In [59]:
def get_num_colors(inp,oup,bl_cols):
    r,c = len(inp),len(inp[0])
    return 

def replace(inp,uni,perm):
    # uni = '234' perm = ['5','7','9']
    #print(uni,perm)
    r_map = { int(c):int(s) for c,s in zip(uni,perm)}
    r,c = len(inp),len(inp[0])
    rp = np.array(inp).tolist()
    #print(rp)
    for i in range(r):
        for j in range(c):
            if(rp[i][j] in r_map):
                rp[i][j] = r_map[rp[i][j]]
    return rp
            
    
def augment(inp,oup,bl_cols):
    cols = "0123456789"
    npr_map = [1,9,72,3024,15120,60480,181440,362880,362880]
    uni = "".join([str(x) for x in np.unique(inp).tolist()])
    for c in bl_cols:
        cols=cols.replace(str(c),"")
        uni=uni.replace(str(c),"")
        
    exp_size = len(inp)*len(inp[0])*npr_map[len(uni)]
    
    mod = floor(exp_size/120000)
    mod = 1 if mod==0 else mod
    
    #print(exp_size,mod,len(uni))
    result = []
    count = 0
    for comb in combinations(cols,len(uni)):
        for perm in permutations(comb):
            count+=1
#            if(count % mod == 0):
            result.append((replace(inp,uni,perm),replace(oup,uni,perm)))
    return(result)
            
    

def get_flips(inp,oup):
    result = []
    flip_ids = []
    inp = np.array(inp)
    oup = np.array(oup)
    if (np.fliplr(inp)).shape == inp.shape:
        result.append((np.fliplr(inp).tolist(),np.fliplr(oup).tolist()))
        flip_ids.append(0)
    if  (np.rot90(np.fliplr(inp),1)).shape == inp.shape:  
        result.append((np.rot90(np.fliplr(inp),1).tolist(),np.rot90(np.fliplr(oup),1).tolist()))
        flip_ids.append(1)
    if  (np.rot90(np.fliplr(inp),2)).shape == inp.shape:
        result.append((np.rot90(np.fliplr(inp),2).tolist(),np.rot90(np.fliplr(oup),2).tolist()))
        flip_ids.append(2)
    if  (np.rot90(np.fliplr(inp),3)).shape == inp.shape:   
        result.append((np.rot90(np.fliplr(inp),3).tolist(),np.rot90(np.fliplr(oup),3).tolist()))
        flip_ids.append(3)
    if  (np.flipud(inp)).shape == inp.shape:    
        result.append((np.flipud(inp).tolist(),np.flipud(oup).tolist()))
        flip_ids.append(4)
    if  (np.rot90(np.flipud(inp),1)).shape == inp.shape:    
        result.append((np.rot90(np.flipud(inp),1).tolist(),np.rot90(np.flipud(oup),1).tolist()))
        flip_ids.append(5)
    if  (np.rot90(np.flipud(inp),2)).shape == inp.shape:   
        result.append((np.rot90(np.flipud(inp),2).tolist(),np.rot90(np.flipud(oup),2).tolist()))
        flip_ids.append(6)
    if  (np.rot90(np.flipud(inp),3)).shape == inp.shape:   
        result.append((np.rot90(np.flipud(inp),3).tolist(),np.rot90(np.flipud(oup),3).tolist()))
        flip_ids.append(7)
    if  (np.fliplr(np.flipud(inp))).shape == inp.shape:   
        result.append((np.fliplr(np.flipud(inp)).tolist(),np.fliplr(np.flipud(oup)).tolist()))
        flip_ids.append(8)
    if  (np.flipud(np.fliplr(inp))).shape == inp.shape:    
        result.append((np.flipud(np.fliplr(inp)).tolist(),np.flipud(np.fliplr(oup)).tolist()))
        flip_ids.append(9)
    return(result)

In [60]:
def get_a_size(task_json):
    return 4;

def get_bl_cols(task_json):
    result = []
    bkg_col = getBkgColor(task_json);
    result.append(bkg_col)
    # num_input,input_cnt,num_output,output_cnt
    met_map = {}
    for i in range(10):
        met_map[i] = [0,0,0,0]
        
    total_ex = 0
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        u,uc = np.unique(inp, return_counts=True)
        inp_cnt_map = dict(zip(u,uc))
        u,uc = np.unique(oup, return_counts=True)
        oup_cnt_map = dict(zip(u,uc))
        
        for col,cnt in inp_cnt_map.items():
            met_map[col][0] = met_map[col][0] + 1
            met_map[col][1] = met_map[col][1] + cnt
        for col,cnt in oup_cnt_map.items():
            met_map[col][2] = met_map[col][2] + 1
            met_map[col][3] = met_map[col][3] + cnt
        total_ex+=1
    
    for col,met in met_map.items():
        num_input,input_cnt,num_output,output_cnt = met
        if(num_input == total_ex or num_output == total_ex):
            result.append(col)
        elif(num_input == 0 and num_output > 0):
            result.append(col)
    
    result = np.unique(result).tolist()
    if(len(result) == 10):
        result.append(bkg_col)
    return np.unique(result).tolist()

In [61]:
#bl_cols = get_bl_cols(task)
#result = augment(task['train'][0]['input'],task['train'][0]['output'],bl_cols)

In [62]:
#result = get_flips(task['train'][0]['input'],task['train'][0]['output'])

In [63]:
def collecttasks(task_json,aug,bl_cols,flip=True):    
    task_dset = []
    for pair in task_json['train']:
        inp,oup = pair["input"],pair["output"]
        inp_big = return_big(inp)
        oup_big = return_big(oup)
        task_dset.append((inp_big,oup_big))
        if(flip):
            flips = get_flips(inp,oup)
            #task_dset += flips
            m = len(flips)
            for i in range(m):
                ainp,aoup = flips[i]
                ainp_big = return_big(ainp)
                aoup_big = return_big(aoup)
                task_dset.append((ainp_big,aoup_big))
                if(aug):
                    augs = augment(ainp,aoup,bl_cols)
                    for x in augs:
                        aug0_big = return_big(x[0])
                        aug1_big = return_big(x[1])
                        task_dset.append((aug0_big,aug1_big))
        else:
            if(aug):
                augs = augment(inp,oup,bl_cols)
                for x in augs:
                    aug0_big = return_big(x[0])
                    aug1_big = return_big(x[1])
                    task_dset.append((aug0_big,aug1_big))
    np.random.shuffle(task_dset)       
    task_dset = task_dset[:10000]
    m = len(task_dset)
    rem = 32 - m%32
    for i in range(rem):
        task_dset.append(task_dset[i])
    task_dset = np.array(task_dset)        
    return(task_dset)


def return_big(img):
    img = np.array(img)
    new_img = np.zeros((30,30),int)
    r,c = img.shape
    for i in range(r):
        for j in range(c):
            new_img[i,j] = img[i,j]
    return(new_img)

In [64]:
#bl_cols = get_bl_cols(task)
#dset = collecttasks(task,True,bl_cols,True)

In [65]:
#dset[:,1].reshape(len(dset),-1).shape

In [66]:
#dl = provider('train',dset)

In [67]:
#batch = next(iter(dl))

## Dataloader

In [68]:
class generic(Dataset):
    
    def __init__(self,task_dset):
        self.data_x = task_dset[:,0]
        self.data_y = task_dset[:,1]
        #self.transform = get_transforms()
        
    def __len__(self):
        return(len(self.data_x))
    
    def __getitem__(self,idx):
        labels = self.data_y[idx]
        labels = np.expand_dims(labels,axis = 0)
        #labels = labels.flatten()
        img1 = self.data_x[idx]
        img = np.expand_dims(img1,axis = 0)
        #b = img.copy()
        #c = img.copy()
        #img = np.concatenate((img,b,c),axis = 2)
        img = np.float32(img)
        labels = np.float32(labels)
        #augmented = self.transform(image = img)
        #img = augmented['image']
        #trfms = Compose([])
        img = torch.tensor(img)
        img = img.cuda()
        labels = torch.tensor(labels)
        labels = labels.cuda()
        #augmented = trfms(image = labels)
        #labels = augmented['image']
        return(img,labels)
    
#def get_transforms():
#    list_transforms = []
#    list_transforms.extend([ToTensor(),])
#    trfms = Compose(list_transforms)
#    return(trfms)    
    

In [69]:
def provider(phase,task_dset,mean=None,std=None,batch_size = 32):
    if phase == 'train':
        image_dataset = generic(task_dset)
        dataloader = DataLoader(image_dataset,batch_size = batch_size)
    else:
        image_dataset = denoiser(dset_df_X_eval,dset_df_Y_eval)
        dataloader = DataLoader(image_dataset,batch_size = batch_size)
    return dataloader

## Model 

In [70]:
import torch
import torch.nn as nn
import torch.utils as utils
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [71]:
# Set Hyperparameters

epoch = 10
batch_size = 32
learning_rate = 0.0005

In [72]:
# Encoder 
# torch.nn.Conv2d(in_channels, out_channels, kernel_size,
#                 stride=1, padding=0, dilation=1,
#                 groups=1, bias=True)
# batch x 1 x 28 x 28 -> batch x 512

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1,32,3,padding=1),   # batch x 16 x 28 x 28
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32,32,3,padding=1),   # batch x 16 x 28 x 28
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32,64,3,padding=1),  # batch x 32 x 28 x 28
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64,64,3,padding=1),  # batch x 32 x 28 x 28
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.MaxPool2d(2,2)   # batch x 64 x 14 x 14
        )
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,128,3,padding=1),  # batch x 64 x 14 x 14
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128,128,3,padding=1),  # batch x 64 x 14 x 14
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.MaxPool2d(2,2),
                        nn.Conv2d(128,256,3,padding=1),  # batch x 64 x 7 x 7
                        nn.ReLU()
        )
        
                
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(batch_size, -1)
        return out

In [73]:
# Decoder 
# torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
#                          stride=1, padding=0, output_padding=0,
#                          groups=1, bias=True)
# output_height = (height-1)*stride + kernel_size - 2*padding + output_padding
# batch x 512 -> batch x 1 x 28 x 28
batch_size = 32

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.layer1 = nn.Sequential(
                        nn.ConvTranspose2d(256,128,3,2,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.ConvTranspose2d(128,128,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.ConvTranspose2d(128,64,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.ConvTranspose2d(64,64,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(64)
        )
        self.layer2 = nn.Sequential(
                        nn.ConvTranspose2d(64,32,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.ConvTranspose2d(32,32,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.ConvTranspose2d(32,1,3,2,0,1),
                        nn.ReLU()
        )
        
    def forward(self,x):
        out = x.view(batch_size,256,7,7)
        out = self.layer1(out)
        out = self.layer2(out)
        #out = out.view(batch_size,900)
        return out

In [74]:
#encoder = Encoder().cuda()
#decoder = decoder().cuda()
#encoder = Encoder()
#decoder = Decoder()

In [75]:
#batch[0].shape

In [76]:
#Check size
#t = decoder(encoder(batch[0]))
#t.shape

In [77]:
#parameters = list(encoder.parameters())+ list(decoder.parameters())
#loss_func = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(parameters, lr=learning_rate)

In [78]:
#def validation_loss(img_batch,labels_batch):
#    m = len(img_batch)
#    agg = 0
#    for i in range(m):
#        img = img_batch[i].detach().numpy()
#        labels = labels_batch[i].detach().numpy()
#        img = img.flatten()
#        labels = labels.flatten()
#        t = np.equal(img,labels)
#        s = np.sum(t)/len(t)
#        agg += s
#    return(agg/m)

In [79]:
#dl = provider('train',res)
#batch = next(iter(dl))

In [80]:
#for i in range(epoch):
#    for batch in tqdm(dl):
#        #image = batch[1].cuda()
#        #batch[0] = batch[0].cuda()
#        image = batch[1]
#        batch[0] = batch[0]
#        optimizer.zero_grad()
#        output = encoder(batch[0])
#        output = decoder(output)
#        loss = loss_func(output,image)
#        loss.backward()
#        optimizer.step()
#    dl_val = provider('val')
#    val_loss = 0
#    count = 0
#    for batch in dl_val:
#        count += 1
#        actual_image = batch[1]
#        optimizer.zero_grad()
#        output = encoder(batch[0])
#        output = decoder(output)
#        #output = np.int(output)
#        #labels = np.int(labels)
#        val_loss += validation_loss(output,actual_image)
#    print('Epoch : '+ str(i))
#    print('Validation Loss : ' + str(val_loss/count))

## Task Solver

In [81]:
## Train with batch_size = 32
def task_solver(task):
    bl_cols = get_bl_cols(task)
    task_dset = collecttasks(task,True,bl_cols,True)
    dl = provider('train',task_dset)
    
    encoder = Encoder().cuda()
    decoder = Decoder().cuda()
    
    
    # Set Hyperparameters

    epoch = 2
    batch_size = 32
    learning_rate = 0.0005
    
    parameters = list(encoder.parameters())+ list(decoder.parameters())
    loss_func = nn.MSELoss()
    optimizer = torch.optim.Adam(parameters, lr=learning_rate)
    
    for i in range(epoch):
        for batch in tqdm(dl):
            #image = batch[1].cuda()
            #batch[0] = batch[0].cuda()
            image = batch[1]
            optimizer.zero_grad()
            output = encoder(batch[0])
            output = decoder(output)
            loss = loss_func(output,image)
            loss.backward()
            optimizer.step()
    test_preds = get_test_preds(task,encoder,decoder)
    return(test_preds)

In [82]:
def get_test_preds(task,encoder,decoder):
    m = len(task['test'])
    test_preds = []
    for i in range(m):
        img1 = task['test'][0]['input']
        img = return_big(img1)
        dset = []
        for i in range(32):
            dset.append(img)
        dset = list(zip(dset,dset))   
        dset = np.array(dset)    
        dl = provider('train',dset)
        batch = next(iter(dl))
        x = encoder(batch[0])
        y = decoder(x)
        y = y.cpu()
        y = y[0][0].detach().numpy()
        y = y.astype(int)
        y = extract_actual_image(task,y)
        test_preds.append(y)
    return(test_preds)

In [83]:
#get_test_preds(task,encoder,decoder)

In [84]:
def convert_to_int(y):
    r,c = y.shape
    for i in range(r):
        for j in range(c):
            y[i,j] = int(y[i,j])
    return(y)        

In [85]:
def extract_actual_image(task,y):
    img = task['train'][0]['input']
    img = np.array(img)
    r,c = img.shape
    new_y = np.zeros((r,c))
    for i in range(r):
        for j in range(c):
            new_y[i,j] = img[i,j]
    return(new_y)        

In [86]:
#task = train_tasks[l_input_otput_same_size[1]]

In [87]:
#t = task_solver(task)

In [88]:
#plot_images([task['test'][0]['input'],task['test'][0]['output'],t[0]])

In [89]:
#img1 = task['test'][0]['input']
#img = return_big(img1)
#dset = []
#for i in range(32):
#    dset.append(img)
#dset = list(zip(dset,dset))   
#dset = np.array(dset)    
#dset = np.expand_dims(dset,axis = 0)
#dset = np.float32(dset)
#dset = torch.tensor(dset)

In [90]:
#dl = provider('train',dset)
#batch = next(iter(dl))

In [91]:
#x = encoder(batch[0])
#y = decoder(x)

In [92]:
#y = y[0][0].detach().numpy()
#y = int(y)

In [93]:
#y.astype(int)

In [94]:
#task_dset = collecttasks(task,True,bl_cols,True)
#encoder = Encoder()
#decoder = Decoder()
#dl = provider('train',dset)

In [95]:
#x = (encoder(img))
#decoder = Decoder(1)
#y = decoder(x)

In [96]:
#batch = next(iter(dl))
#x = decoder(encoder(batch[0]))

## Inference

In [97]:
def inp_oup_dim_same(task_json):
    return all([ len(pair["input"]) == len(pair["output"]) and len(pair["input"][0]) == len(pair["output"][0]) for pair in task_json['train']])

In [98]:
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred 
#
#def combine_preds(tid,pm1,pm3,pm5):
#    result = []
#    for i in range(len(pm1)):
#        tk_s = tid+"_"+str(i)
#        str_pred = flattener(pm1[i])+" "+flattener(pm3[i])+" "+flattener(pm5[i])
#        #print(tk_s,str_pred)
#        result.append([tk_s,str_pred])
#    return result

In [99]:
#x = test_tasks.index[0]
#test_tasks[x]

In [100]:
def dumb_predict(task):
    preds = []
    for pair in task['test']:
        preds.append([[0,0],[0,0]])
    return(preds)

In [101]:
import time 

start = time.time()
result = []
#tr = [] 
for idx in test_tasks.index:
    now = time.time()
    task = test_tasks[idx]
    tk_id = idx
    if now - start <= 7000:
        if inp_oup_dim_same(task):
            t = task_solver(task)
        else:
            t = dumb_predict(task)
    else:
        t = dumb_predict(task)
       
    for i in range(len(t)):
            tk_s = tk_id +"_" + str(i)
            temp = []
            for row in t[i]:
                row = list(row)
                temp.append(row)
            t[i] = temp 
            t[i] = np.array(t[i]).astype(int)
            t[i] = t[i].tolist()
            #tr.append(t[i])
            str_pred = flattener(t[i])
            #print(tk_s,str_pred)
            result.append([tk_s,str_pred])    
        
task_ids = []
task_preds = []
for i in range(len(result)):
    task_ids.append(result[i][0])
    task_preds.append(result[i][1])   

100%|██████████| 2/2 [00:00<00:00, 43.44it/s]


In [102]:
#def calk_score(task_test, predict):
#    return [int(np.equal(sample['output'], pred).all()) for sample, pred in zip(task_test, predict)]

In [103]:
#def get_pred_list(pred_map):
#    l = list(pred_map.keys())
#    l.sort()
#    preds = []
#    for i in l:
#        preds.append(pred_map[i])
#    return(preds)    

In [104]:
#def inp_oup_dim_same(task_json):
#    return all([ len(pair["input"]) == len(pair["output"]) and len(pair["input"][0]) == len(pair["output"][0])
#                for pair in task_json['train']])
#    
#
#solved_task = 0
#total_task = 0
#task_ids = []
#task_preds = []
##total_score_1 = 0
##total_score_2 = 0
##total_score_3 = 0
#
#for task_path in test_path.glob("*.json"):
#    task_json = json.load(open(task_path))
#    tk_id = str(task_path).split("/")[-1].split(".")[0]
#    print(tk_id)
#    if(inp_oup_dim_same(task_json)):
#        a_size = get_a_size(task_json)
#        bl_cols = get_bl_cols(task_json)
#        
#        isflip = False
#        X1,Y1 = gettaskxy(task_json,True,1,bl_cols,isflip)
#        X3,Y3 = gettaskxy(task_json,True,3,bl_cols,isflip)
#        X5,Y5 = gettaskxy(task_json,True,5,bl_cols,isflip)
#        
#        model_1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X1, Y1)
#        model_3 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X3, Y3)
#        model_5 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X5, Y5)
#        
#        pred_map_1 = submit_predict(task_json,model_1,1,False)
#        pred_map_3 = submit_predict(task_json,model_3,3,False)
#        pred_map_5 = submit_predict(task_json,model_5,5,False)
#        
#        #preds_1 = get_pred_list(pred_map_1)
#        #preds_2 = get_pred_list(pred_map_3)
#        #preds_3 = get_pred_list(pred_map_5)
#        #
#        #total_score_1 += calk_score(task_json['test'], preds_1)[0]
#        #total_score_2 += calk_score(task_json['test'], preds_2)[0]
#        #total_score_3 += calk_score(task_json['test'], preds_3)[0]
#        
#        for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_3,pred_map_5):
#            task_ids.append(tks)
#            task_preds.append(str_pred)
#            #print(tks,str_pred)
#        solved_task+=1
#        #break
#    else:
#        pred_map_1 = dumb_predict(task_json)
#        pred_map_3 = dumb_predict(task_json)
#        pred_map_5 = dumb_predict(task_json)
#        
#        for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_3,pred_map_5):
#            task_ids.append(tks)
#            task_preds.append(str_pred)
#            #print(tks,str_pred)
#        
#    total_task+=1

In [105]:
sub_df = pd.DataFrame({"output_id":task_ids,'output':task_preds})
sub_df.to_csv("submission.csv", index=None)

In [106]:
sub_df.iloc[5,:]

output_id                                           1c56ad9f_0
output       |00000000000000|00000000000000|00000000000000|...
Name: 5, dtype: object